# Theoretical Basis

`PyCBA` is based on the matrix stiffness method. 

## Note
Until this section is completed, please also see the Matrix Stiffness Method lecture notes [here](http://www.colincaprani.com/structural-engineering/courses/structural-analysis-iv) for more information.

## Elements
The stiffness matrices for the elements used in `PyCBA` are given below.

Note that the releases relate only to internal hinges within the elements themselves, and not to the supports conditions. Note also, that in modelling joints, there must always be one element with a 'fixed' end at a joint; otherwise the resulting global stiffness matrix will be singular.

### Type 1: Fixed-Fixed
$$
\bf{K} = \frac{EI}{L^3} 
\begin{bmatrix}
12 & 6L & -12 & 6L \\
6L & 4L^2 & -6L & 2L^2 \\
-12 & -6L & 12 & -6L \\
6L & 2L^2 & -6L & 4L^2
\end{bmatrix}
$$

### Type 2: Fixed-Pinned
$$
\bf{K} = \frac{3EI}{L^3} 
\begin{bmatrix}
1 & L & -1 & 0 \\
L & L^2 & -L & 0 \\
-1 & -L & 1 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

### Type 3: Pinned-Fixed
$$
\bf{K} = \frac{3EI}{L^3} 
\begin{bmatrix}
1 & 0 & -1 & L \\
0 & 0 & 0 & 0 \\
-1 & 0 & 1 & -L \\
L & 0 & -L & L^2
\end{bmatrix}
$$

### Type 4: Pinned-Pinned
$$
\bf{K} = 
\begin{bmatrix}
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0
\end{bmatrix}
$$

## Displacements

Displacement along an element are found using cumulative trapezoidal integration to approximate the integral. It is not accurate to use the beam element shape functions due to the presence of inter-nodal loading. Thus, the rotations along an element are found by integrating the curvatures:

$$
\theta(x) = \theta_0 + \int_{x=0}^{x=L} \frac{M(x)}{EI} dx,
$$

and similarly the displacements then found by integrating the rotations:

$$
\delta(x) = \delta_0 + \int_{x=0}^{x=L} \theta(x) dx.
$$

In both cases we approximate the integral with:

$$
\int_0^L y(x) dx \approx \sum_{i=1}^{i=n} \frac{y_{i-1}+y_i}{2} \Delta x.
$$

which is implemented using `SciPy`'s [cumulative trapezoidal](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.cumulative_trapezoid.html#scipy.integrate.cumulative_trapezoid) integration along the $n$ integration points along the member.

Note that $\theta_0$ and $\delta_0$ are required: the rotation and translation at the $i$-node of the element. This is typically the output of the global system of equations; i.e., the nodal displacements. However, for elements with releases (types 2, 3, 4 above), we need to determine the starting rotation, $\theta_0$. From the slope deflection equations we can find this starting rotation at the $i$-node:

$$
\theta_i = \frac{\delta_j - \delta_i}{L} - \frac{L}{3EI}\left(-FEM_i + 0.5FEM_j + M_i - 0.5 M_j\right)
$$

in which the $FEM$ are the consistent nodal loading moments due to the inter-nodal loads, and the $M$ moments are solved from the global stiffness equation.